In [1]:
pip install pandas pyodbc

  Using cached pyodbc-5.2.0-cp311-cp311-win_amd64.whl.metadata (2.8 kB)
Using cached pyodbc-5.2.0-cp311-cp311-win_amd64.whl (68 kB)
Note: you may need to restart the kernel to use updated packages.



[notice] A new release of pip is available: 24.0 -> 24.3.1
[notice] To update, run: C:\Users\mariu\AppData\Local\Microsoft\WindowsApps\PythonSoftwareFoundation.Python.3.11_qbz5n2kfra8p0\python.exe -m pip install --upgrade pip


In [1]:
import pandas as pd
import pyodbc
import numpy as np

In [2]:
# Leer el archivo CSV
dfPurchasePrices = pd.read_csv("2017PurchasePricesDec.csv")

In [3]:
dfPurchasePrices.head()

,Brand,Description,Price,Size,Volume,Classification,PurchasePrice,VendorNumber,VendorName
0,58,Gekkeikan Black & Gold Sake,12.99,750mL,750,1,9.28,8320,SHAW ROSS INT L IMP LTD
1,62,Herradura Silver Tequila,36.99,750mL,750,1,28.67,1128,BROWN-FORMAN CORP
2,63,Herradura Reposado Tequila,38.99,750mL,750,1,30.46,1128,BROWN-FORMAN CORP
3,72,No. 3 London Dry Gin,34.99,750mL,750,1,26.11,9165,ULTRA BEVERAGE COMPANY LLP
4,75,Three Olives Tomato Vodka,14.99,750mL,750,1,10.94,7245,PROXIMO SPIRITS INC.


In [4]:
dfPurchasePrices.shape

(12261, 9)

In [5]:
dfPurchasePrices.isnull().sum()

Brand             0
Description       1
Price             0
Size              1
Volume            1
Classification    0
PurchasePrice     0
VendorNumber      0
VendorName        0
dtype: int64

In [6]:
# Mostrar las columnas con valores nulos por fila
for index, row in dfPurchasePrices.iterrows():
    null_columns = row[row.isnull()].index
    if len(null_columns) > 0:
        print(f"Fila {index} tiene valores nulos en las columnas: {', '.join(null_columns)}")

Fila 7915 tiene valores nulos en las columnas: Description, Size, Volume


In [7]:
# como es solo una fila la elimino
dfPurchasePrices = dfPurchasePrices.dropna()

In [8]:
# verifico
dfPurchasePrices.isnull().sum()

Brand             0
Description       0
Price             0
Size              0
Volume            0
Classification    0
PurchasePrice     0
VendorNumber      0
VendorName        0
dtype: int64

In [9]:
dfPurchasePrices.shape

(12260, 9)

In [10]:
import pandas as pd
import pyodbc


# Conectar a SQL Server (sin contraseña)
conn = pyodbc.connect('DRIVER={SQL Server};'
                      'SERVER=MARIUGENIAIT\SQLEXPRESS;'
                      'DATABASE=LiquorDistributorDB;'
                      'Trusted_Connection=yes;')  # Usa Trusted_Connection si no tienes usuario/contraseña

cursor = conn.cursor()

# Insertar los datos en la tabla
for index, row in dfPurchasePrices.iterrows():
    try:
        cursor.execute("""
            INSERT INTO PurchasePrices (Brand, Description, Price, Size, Volume, Classification, PurchasePrice, VendorNumber, VendorName)
            VALUES (?, ?, ?, ?, ?, ?, ?, ?, ?)
        """, 
        row['Brand'], 
        row['Description'], 
        float(row['Price']) if pd.notnull(row['Price']) else None,  # Asegurar que Price sea decimal o NULL
        row['Size'], 
        float(row['Volume']) if pd.notnull(row['Volume']) else None,  # Asegurar que Volume sea decimal o NULL
        row['Classification'], 
        float(row['PurchasePrice']) if pd.notnull(row['PurchasePrice']) else None,  # Asegurar que PurchasePrice sea decimal o NULL
        int(row['VendorNumber']) if pd.notnull(row['VendorNumber']) else None,  # Asegurar que VendorNumber sea int o NULL
        row['VendorName']
        )
    except Exception as e:
        print(f"Error en la fila {index}: {e}")  # Mostrar errores de inserción y continuar con las demás filas

# Confirmar los cambios y cerrar la conexión
conn.commit()
conn.close()

print("Datos importados exitosamente.")


Error en la fila 542: could not convert string to float: 'Unknown'
Error en la fila 5921: could not convert string to float: 'Unknown'
Error en la fila 8795: could not convert string to float: 'Unknown'
Error en la fila 10009: could not convert string to float: 'Unknown'
Datos importados exitosamente.


Por el resultado anterior, borré la tabla en sql server. Y ahora reemplazaré Unknown por 0

In [24]:
# Reemplazar todos los valores "Unknown" por 0 en todo el DataFrame
dfPurchasePrices.replace('Unknown', 0, inplace=True)

In [11]:
# Verificar si "Unknown" aparece en alguna celda de cualquier columna
mask = dfPurchasePrices.applymap(lambda x: 'Unknown' in str(x))

# Ver las filas y columnas donde aparece "Unknown"
unknown_rows = dfPurchasePrices[mask.any(axis=1)]

# Mostrar las filas donde aparece "Unknown"
print(unknown_rows)

       Brand                   Description  Price     Size   Volume  \
542     2993             Angostura Bitters   7.49  Unknown  Unknown   
5396   16398  Shane The Unknown Syrah No C  39.99    750mL      750   
5921    9908      Tito's Copper Mug 2 Pack  21.01  Unknown  Unknown   
8795    8992                      Group 92   1.99  Unknown  Unknown   
10009  90590  Overture Champagne 2Glass Pk  19.95  Unknown  Unknown   

       Classification  PurchasePrice  VendorNumber  \
542                 1           5.39          5895   
5396                2          25.80          7749   
5921                1          16.15          4425   
8795                1           1.43          1703   
10009               2          13.12          4425   

                                VendorName  
542            Mizkan Americas, Inc.        
5396           R.P.IMPORTS INC              
5921                 MARTIGNETTI COMPANIES  
8795   ALISA CARR BEVERAGES                 
10009                MA

In [12]:
# Otra forma de Reemplazar valores no válidos con NaN
dfPurchasePrices.replace(["Unknown", "#N/A", "NA"], np.nan, inplace=True)

In [ ]:
# una forma de verificar que en las columnas numericas hay solo numeros:
# Convertir columnas a tipos de datos correctos
df['Price'] = pd.to_numeric(df['Price'], errors='coerce')  # Convertir a float
df['Volume'] = pd.to_numeric(df['Volume'], errors='coerce')
df['PurchasePrice'] = pd.to_numeric(df['PurchasePrice'], errors='coerce')
df['VendorNumber'] = pd.to_numeric(df['VendorNumber'], errors='coerce', downcast='integer')
# errors='coerce' reemplaza valores no convertibles con NaN.

In [ ]:
# eliminar filas con valores nulos:
df.dropna(subset=['Price', 'PurchasePrice'], inplace=True)  # Eliminar si Price o PurchasePrice son NaN
# Reemplazar faltantes con un valor:
df['Price'].fillna(0, inplace=True)  # Rellenar NaN con 0
df['VendorName'].fillna('Desconocido', inplace=True)  # Rellenar texto faltante

In [14]:
duplicados= dfPurchasePrices.duplicated().sum()
duplicados

0

In [ ]:
# eliminar duplicados
df.drop_duplicates(inplace=True) 

In [ ]:
# validar datos antes de insertarlos:
# Filtrar registros con precios negativos o nulos
df = df[(df['Price'] > 0) & (df['PurchasePrice'] > 0)]

# Filtrar registros con valores inesperados
df = df[df['Volume'] <= 10000]  # Volumen no mayor a 10,000 es un ejemplo


In [26]:
# vuelvo a llenar la tabla
import pandas as pd
import pyodbc


# Conectar a SQL Server (sin contraseña)
conn = pyodbc.connect('DRIVER={SQL Server};'
                      'SERVER=MARIUGENIAIT\SQLEXPRESS;'
                      'DATABASE=LiquorDistributorDB;'
                      'Trusted_Connection=yes;')  # Usa Trusted_Connection si no tienes usuario/contraseña

cursor = conn.cursor()

# Insertar los datos en la tabla
for index, row in dfPurchasePrices.iterrows():
    try:
        cursor.execute("""
            INSERT INTO PurchasePrices (Brand, Description, Price, Size, Volume, Classification, PurchasePrice, VendorNumber, VendorName)
            VALUES (?, ?, ?, ?, ?, ?, ?, ?, ?)
        """, 
        row['Brand'], 
        row['Description'], 
        float(row['Price']) if pd.notnull(row['Price']) else None,  # Asegurar que Price sea decimal o NULL
        row['Size'], 
        float(row['Volume']) if pd.notnull(row['Volume']) else None,  # Asegurar que Volume sea decimal o NULL
        row['Classification'], 
        float(row['PurchasePrice']) if pd.notnull(row['PurchasePrice']) else None,  # Asegurar que PurchasePrice sea decimal o NULL
        int(row['VendorNumber']) if pd.notnull(row['VendorNumber']) else None,  # Asegurar que VendorNumber sea int o NULL
        row['VendorName']
        )
    except Exception as e:
        print(f"Error en la fila {index}: {e}")  # Mostrar errores de inserción y continuar con las demás filas

# Confirmar los cambios y cerrar la conexión
conn.commit()
conn.close()

print("Datos importados exitosamente.")

Datos importados exitosamente.


In [27]:
# Verificar si hay tipo de datos diferentes a numeros en las columnas numericas
# Especificar las columnas que deseas verificar
columns_to_check = ['Price', 'PurchasePrice', 'Volume', 'VendorNumber']  # Ejemplo de columnas

# Verificar si hay valores no numéricos en las columnas especificadas
invalid_rows = dfPurchasePrices[~dfPurchasePrices[columns_to_check].apply(pd.to_numeric, errors='coerce').notnull().all(axis=1)]

# Mostrar las filas con datos incorrectos en alguna de las columnas
print(invalid_rows)

Empty DataFrame
Columns: [Brand, Description, Price, Size, Volume, Classification, PurchasePrice, VendorNumber, VendorName]
Index: []


CONTINUAR LLENANDO LAS OTRAS TABLAS